## Record Audio

In [26]:
import wave
import keras
import pyaudio
import pandas as pd
import numpy as np

import IPython.display as ipd

In [27]:
CHUNK = 1024 
FORMAT = pyaudio.paInt16 #paInt8
CHANNELS = 2 
RATE = 44100 #sample rate
RECORD_SECONDS = 6
WAVE_OUTPUT_FILENAME = "demo_audio1.wav"
emotions=["Anger","disgust","fear","happy","Neutral", "sad", "surprise"]

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK) #buffer

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data) # 2 bytes(16 bits) per channel

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording


KeyboardInterrupt: 

## Load The SER Model

In [28]:
# loading json and creating model
from keras.models import model_from_json
json_file = open('./utils/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./Trained_Models/Speech_Emotion_Recognition_Model.h5")
print("Loaded model from disk")

Loaded model from disk


In [50]:
demo_audio_path = './demo_audio/demo_audio_4.wav'
# ipd.Audio(demo_audio_path)

In [51]:
from utils.feature_extraction import get_audio_features
demo_mfcc, demo_pitch, demo_mag, demo_chrom = get_audio_features(demo_audio_path,20000)

In [52]:
mfcc = pd.Series(demo_mfcc)
pit = pd.Series(demo_pitch)
mag = pd.Series(demo_mag)
C = pd.Series(demo_chrom)
demo_audio_features = pd.concat([mfcc,pit,mag,C],ignore_index=True)

In [53]:
demo_audio_features= np.expand_dims(demo_audio_features, axis=0)
demo_audio_features= np.expand_dims(demo_audio_features, axis=2)

In [54]:
demo_audio_features.shape

(1, 65, 1)

In [55]:
livepreds = loaded_model.predict(demo_audio_features, 
                         batch_size=32, 
                         verbose=1)

1/1 [==============================] - 0s 134ms/step


In [56]:
livepreds

array([[7.7023834e-02, 4.3669675e-02, 8.6259234e-01, 4.2046946e-03,
        2.0782991e-04, 6.7283232e-03, 5.5732066e-03]], dtype=float32)

In [57]:
#emotions=["anger","disgust","fear","happy","neutral", "sad", "surprise"]
index = livepreds.argmax(axis=1).item()
index

2

In [58]:
emotions[index]

'fear'